## TWITTER Happyness Analysys

Twitter is one of the most popular microblogging sites. Twitter users create “tweets” (status messages) about different topics based on their opinion. The purpose of this project is to automatically analyse the sentiment (positive or negative) of tweet. This is very useful because it allows feedback to aggregate without manual intervention.  
• Organizations can also use this to gather critical feedback about problems in newly released products.

• Marketers can use this to research public opinion of their company and products, or to analyse customer satisfaction. 

• Consumers can use sentiment analysis to research products or services before making a purchase. 

####  Connecting to Twitter

Connecting to the twitter API using tweepy (twitter for python). Tweepy supports accessing Twitter via method, OAuth

In [79]:
import tweepy, twitter_text
import csv
import pandas as pd
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import preprocessor as p
import json
from itertools import chain
import objectpath
import re

import matplotlib.pyplot as plt
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'
pd.set_option("display.max_colwidth", 10000)
pd.option_context('display.colheader_justify','left')

To connetct to Twitter, use your own credentials:

In [ ]:
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_token = 'XXX'
access_token_secret = 'XXX'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)
print(api)

#### Searching for jobcentre hashtag

In [ ]:
no_queries = 1000
tweets1 = tweepy.Cursor(api.search, q="#jobcentre", 
                        lang="en",
                        tweet_mode='extended',
                        since="2017-01-01").items(no_queries)
tweets2 = tweepy.Cursor(api.search, q="@Jobcentre", 
                        lang="en",
                        tweet_mode='extended',
                        since="2017-01-01").items(no_queries)
tweets3 = tweepy.Cursor(api.search, q="#dwp", 
                        lang="en",
                        tweet_mode='extended',
                        since="2017-01-01").items(no_queries)

In [ ]:
tweets = chain(tweets1, tweets2, tweets3)

In [ ]:
# for tweet in tweets:
#     print(tweet)

#### Cleaning tweets 

Twitter “tweets” contain  mainly three parts : 

➢ Screen name : User who tweets or retweets 

➢ Text : Main message conveyed by tweet 

➢ Hashtags : Categorial tags, used to highlight content in tweet

Our analysis is basically on the text part of tweet, hashtags can also contribute to this analysis (without “#”) to some extent. Screen names are of no use because they are just user names and doesn’t depict any sentiments. So they just add on to computation and work. Here comes the need of Cleaning our tweets for better analysis. 

In [ ]:
my_list_of_dicts = []
for json_tweet in tweets:
    my_list_of_dicts.append(json_tweet._json)

In [ ]:
with open('jobcentre.txt', 'w') as file:   
        file.write(json.dumps(my_list_of_dicts, indent=4))

In [65]:
def tweetdf(json_txt):
    ''' Read text file containing tweets in json format & create dataframe
    Fields
    id
    full_text
    
    ''' 
    my_demo_list = []
    with open(json_txt, encoding='utf-8') as json_file:  
        all_data = json.load(json_file)
        for each_dictionary in all_data:
            tweet_id = each_dictionary['id']
            full_text = each_dictionary['full_text']
    #         favorite_count = each_dictionary['favorite_count']
    #         retweet_count = each_dictionary['retweet_count']
            created_at = each_dictionary['created_at']
            my_demo_list.append({'tweet_id': str(tweet_id).strip(),
                                 'full_text': str(full_text).strip().replace(r'\u',''),
    #                              'favorite_count': int(favorite_count),
    #                              'retweet_count': int(retweet_count),
                                 'created_at': created_at,
                                })
    tweet_df = pd.DataFrame(my_demo_list, columns = 
                              ['tweet_id', 'full_text', 
    #                            'favorite_count', 'retweet_count', 
                               'created_at'])
    return tweet_df

In [66]:
tweet_df = tweetdf('jobcentre.txt')
tweet_df['full_text'] = tweet_df['full_text'].astype(str).str.lower()
tweet_df = tweet_df[~tweet_df['full_text'].str.contains("brexit")]
tweet_df = tweet_df[~tweet_df['full_text'].str.contains("farage")]
no_queries = tweet_df.count()[1]
no_queries

1038

In [78]:
tweet_df.head()

,tweet_id,full_text,created_at,happiness_rank
0,1109372701842513921,rt @davidbruce1982: half term at the new life centre. just some of our projects to make a note of. #open #community #p3 #cab #jobcentre #y2026,Sat Mar 23 08:33:57 +0000 2019,102058.0
2,1109124922557763584,half term at the new life centre. just some of our projects to make a note of. #open #community #p3 #cab #jobcentre #youth #spilsby https://t.co/29jqkavxhq,Fri Mar 22 16:09:22 +0000 2019,212135.0
3,1108880385344180224,forgot to tell u lot this but i went to jobcentre today n mi mate jake thoght it wuld b funnay to put fookin spice in mi folda with mi papers ! shat meself but thnx god i noticed d83dde21d83dde21d83dde21 rlly taking da piss watch out for dis guy if u kno him #jobcentre #takesdapiss #wtf,Thu Mar 21 23:57:40 +0000 2019,101917.0
4,1108847756918038528,"""we have engaged with social action but we haven't yet rediscovered a vision to build a church for the poor. this needs to change"" - a church for the poor https://t.co/jstq1spbtc #social action #poor #church #uk #universalcredit #dwp #jobcentre #mps #housing #homelessness https://t.co/yyjd65bwpf",Thu Mar 21 21:48:01 +0000 2019,194890.0
6,1108484383219036160,recieved this letter from #dwp regards to my #jobcentre interview on 15/3/19 for my claim on #universalcredit check the bottom who it's signed off by. they do not give a damn about people's lives or welfare. four months and counting. #isolationbooth #bulletproofglass https://t.co/jf7m3haq5z,Wed Mar 20 21:44:06 +0000 2019,174947.0


#### Calculating  polarity scores
To calculate Polarity, Vader Algorithm is used. The Vader Algorithm outputs sentiment scores to 4 classes of sentiments:

• neg   Negative 

• pos   Positive

• neu   Neutral

• Compound : Compound 
Using these sentiment scores we can easily calculate percentage classification of Sentiment on tweets fetched. 
 

In [ ]:
def percent(part,whole):
    return(100* float(part)/float(whole))

positive = 0
negative = 0
neutral = 0

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# for i in list1:
for i in tweet_df['full_text']:
    sps = sid.polarity_scores(i)
    negative +=sps['neg']
    neutral +=sps['neu']
    positive +=sps['pos']
  
positive = round(percent(positive,no_queries),2)
negative = round(percent(negative,no_queries),2)
neutral = round(percent(neutral,no_queries),2)

print(positive,'%'+' positive')
print(negative,'%'+' negative')
print(neutral,'%'+' neutral')

###  Visual Interpretation of Analysis 

To visually interpret our sentiment analysis, we will further use Matplotlib library of Python for plotting a Pie Chart, that diagrammatically shows people opinion on the searched query. 

In [ ]:
labels = ['Positive ['+str(positive)+'%]','Neutral ['+str(neutral)+'%]','Negative ['+str(negative)+'%]']
sizes = [positive,neutral,negative]
colors = ['green','blue','red']
patches,texts  =plt.pie(sizes,colors = colors,startangle=90)

plt.axis('equal')
plt.legend(patches,labels,loc='best')
plt.title('Analysis for jobcentre on '+str(no_queries)+' tweets')
plt.tight_layout()
plt.show()

#### Importing dataset

Dataset twitter_samples used here contain 3 files : 

1 negative_tweets.json (Contain sample negative tweets)

2 positive_tweets,json  (Contain sample positive tweets) 

3 tweets.20150430-223406.json (Contain sample neutral tweets) 




In [ ]:
from nltk.corpus import twitter_samples
twitter_samples.fileids()

In [ ]:
negtweets = twitter_samples.strings('negative_tweets.json')
postweets = twitter_samples.strings('positive_tweets.json')
negtweets = twitter_samples.strings('negative_tweets.json')
postweets = twitter_samples.strings('positive_tweets.json')

#### Training classifier object with dataset

In order to classify tweets on the basis of sentiments, we need to train a classifier using Machine learning methods. Some of popular methods are Support Vector Machines (SVM), Naive Bayes Classifier etc. Naïve Bayes Classifier being the most convenient one, is used here to train  twitter_samples (nltk corpora) 


In [ ]:
def word_feats(words):
    return dict([(word,True) for word in words])

negfeats = [(word_feats(nltk.word_tokenize(p.clean(i))),'neg') for i in negtweets]
posfeats = [(word_feats(nltk.word_tokenize(p.clean(i))),'pos') for i in postweets]
#print(negfeats)


#neg_t = word_feats(negfeats,neg_words)
#pos_t = word_feats(posfeats,pos_words)
negcutoff = int(len(negfeats)*1/2)
poscutoff = int(len(posfeats)*1/2)
trainfeats = negfeats[:negcutoff]+posfeats[:poscutoff]
testfeats = posfeats[negcutoff:]+posfeats[poscutoff:]

print("Train on %d instances, test on %d instances"%(len(trainfeats),len(testfeats)))
classifier = NaiveBayesClassifier.train(trainfeats)

### CALCULATING ACCURACY OF CLASSIFIER OBJECT

In [ ]:
print("Accuracy:",nltk.classify.util.accuracy(classifier,testfeats))

Train on 6666 instances , test on 3334     instances
Accuracy: 0.73125374925015

Train on 7500 instances , test on 2500     instances
Accuracy: 0.7208

Here we saw that with increase in training examples accuracy decreases. It is clear case of High Bias. So to improve accuracywe can do following:

1. Decrease Training Examples .
2. Switching to some other ML model for classification


### PREDICTING SENTIMENT FROM TWEETS FETCHED

In [ ]:
for i in tweet_df['full_text'][:20]:
    test_tweet = word_feats(i)
    print(i+"::"+classifier.classify(test_tweet))

#### Most informative features

In [ ]:
classifier.show_most_informative_features()

### Happiness Analysis

Using the labMIT Happness index.

I modified the original list to take into account that `benefit` is a neutral word in the context of DWP / Jobcentres

In [70]:
labMIT = pd.read_csv("labMIT-1.0-mod.txt",  delim_whitespace=True)
labMIT.head()

,word,happiness_rank,happiness_average,happiness_standard_deviation,twitter_rank,google_rank,nyt_rank,lyrics_rank
0,laughter,1,8.50,0.9313,3600,--,--,1728
1,happiness,2,8.44,0.9723,1853,2458,--,1230
2,love,3,8.42,1.1082,25,317,328,23
3,happy,4,8.30,0.9949,65,1372,1313,375
4,laughed,5,8.26,1.1572,3334,3542,--,2332


In [ ]:
labMIT.tail()

In [71]:
happiness_rank = []
for i in tweet_df['full_text']:
    tokens = nltk.word_tokenize(i)
    cnt = 0
    for j in labMIT.index:
        val = labMIT.at[j,'word']
        if (val in tokens):
            cnt = cnt + labMIT.at[ j,'happiness_rank']

    happiness_rank.append(cnt) 
print(len(happiness_rank))
print(len(tweet_df))

1038
1038


In [72]:
tweet_df.shape

(1038, 3)

In [73]:
tweet_df['happiness_rank'] = pd.Series(happiness_rank)

In [74]:
twdf = tweet_df.dropna(subset=['happiness_rank'])

In [80]:
twdf = twdf.sort_values('happiness_rank')
twdf[['full_text','happiness_rank'] ].head()

,full_text,happiness_rank
554,rt @respectisvital: d83ddea8new - #gogglebox pushes dwp propaganda in planned broadcast.\n\ntonight saw @channel4 sell their soul after they aired a2026,11581.0
78,"rt @mrtopple: umm, #isitok that while nearly 4m people have signed the #revokearticle50petition only 7k have signed one calling for an inqu2026",11941.0
1029,rt @hovellinghermit: how can #disabled people have faith in a system where paperwork submitted by the #dwp in a benefits appeal tribunal ca2026,13268.0
12,come along to our stand at gloucester job centre today and discuss your future employment - rich &amp; shell are waiting to speak to you until noon today d83ddc4dd83ddc4d\n#gloshour #jobcentre #gloucester @jcpingloucester https://t.co/ojczegawue,13759.0
488,"rt @mrtopple: umm, #isitok that while nearly 4m people have signed the #revokearticle50petition only 7k have signed one calling for an inqu2026",17055.0


In [81]:
twdf[['full_text', 'happiness_rank']].tail(10)

,full_text,happiness_rank
1016,"rt @jordaniel_rb: @hovellinghermit then whomever wrote that particular statement of the apellant, should be sought and employment terminate2026",180599.0
1024,rt @hovellinghermit: how can #disabled people have faith in a system where paperwork submitted by the #dwp in a benefits appeal tribunal ca2026,185263.0
744,"rt @mrtopple: umm, #isitok that while nearly 4m people have signed the #revokearticle50petition only 7k have signed one calling for an inqu2026",187752.0
1011,dwp document refers to benefit claimant as 'lying bitch'\n\nhttps://t.co/ejwh1srpyl\nthis is the reason so many hate the #dwp and their bullying of #vulnerablepeople \n#dwpcrimes\n@amberruddhr,192979.0
4,"""we have engaged with social action but we haven't yet rediscovered a vision to build a church for the poor. this needs to change"" - a church for the poor https://t.co/jstq1spbtc #social action #poor #church #uk #universalcredit #dwp #jobcentre #mps #housing #homelessness https://t.co/yyjd65bwpf",194890.0
362,rt @mojos55: #dwp unqualified assessors.. having power to over ride our doctors &amp; consultants diagnosis ! if i were the patient doctor or c2026,196949.0
785,@raging545 @mrtopple @pressjournal stick to the argument.\n\nwhat's more important. \n\nthe #eu or #dwp killing 30 people a day?,203845.0
2,half term at the new life centre. just some of our projects to make a note of. #open #community #p3 #cab #jobcentre #youth #spilsby https://t.co/29jqkavxhq,212135.0
916,rt @johnpringdns: the disabled daughter of a woman who took her own life after losing her disability benefits has explained why she believe2026,214371.0
25,banned from the #jobcentre - after 'losing the plot' over months of trying to get #universalcredit #hostileenviroment #homelessness https://t.co/gprpfysg2w,228860.0


In [82]:
string = r'\uword'
string.replace(r'\u','')
string

'\\uword'

In [88]:
string = 'xyz \u2026'
string.replace(r'\u2026','')
string

'xyz …'

In [85]:
string = r'\uword'
string.replace(r'\u', '')
string

'\\uword'